In [6]:
!pip install opencv-python

/bin/bash: /home/latai/anaconda3/envs/latai/lib/python3.7/site-packages/cv2/../../../../lib/libtinfo.so.6: no version information available (required by /bin/bash)
     |████████████████████████████████| 60.5 MB 520 kB/s  eta 0:00:01


In [2]:
!pip install torch

     |████████████████████████████████| 750.6 MB 39 kB/s s eta 0:00:01     |██████████▋                     | 249.8 MB 10.7 MB/s eta 0:00:47     |███████████████▊                | 369.3 MB 8.3 MB/s eta 0:00:46


In [1]:
import numpy as np
import cv2
from numpy.linalg import inv
from torch import Tensor
from torch.autograd import Variable
import torch
import torch.nn.functional as F

In [2]:
# Get the indices matrix refer Testpy_numpy.ipynb
pixel_coordinate = np.indices([320, 256]).astype(np.float32)
# Concatenate the pixel coordinate matrix ([2, 320, 256]) with the ones matrix [1, 320, 256] and the resultant matrix is [3, 320, 256]
pixel_coordinate = np.concatenate(
    (pixel_coordinate, np.ones([1, 320, 256])), axis=0)
# Reshape refer Testpy_numpy.ipynb
pixel_coordinate = np.reshape(pixel_coordinate, [3, -1])

In [3]:
gt_poses = []

# Get the extrinsic parameters i,e the position of the camera while capturing the images, rotation matrix and translation.
with open('/home/latai/Documents/Master_thesis/Data/seq-01_formatted/poses.txt') as f:
    for l in f.readlines():
        l = l.strip('\n')
        gt_poses.append(np.array(l.split(' ')).astype(np.float32).reshape(4, 4))

In [4]:
# next image pose
r_pose = gt_poses[1]
# current image pose
n_pose = gt_poses[0]

# read the next frame image 
r_img = cv2.imread("/home/latai/Documents/Master_thesis/Data/seq-01_formatted/images/0001.png")
# read the current frame image
n_img = cv2.imread("/home/latai/Documents/Master_thesis/Data/seq-01_formatted/images/0000.png")

K = np.loadtxt('/home/latai/Documents/Master_thesis/Data/seq-01_formatted//K.txt').astype(np.float32).reshape((3, 3))

In [5]:
left_pose = r_pose
right_pose = n_pose
print(left_pose)

[[ 0.8744711  -0.33191165  0.3535954   0.33358625]
 [ 0.33531535  0.9405307   0.05358316  0.1364616 ]
 [-0.3503717   0.07171355  0.933794   -0.53893393]
 [ 0.          0.          0.          1.        ]]


In [6]:
print(inv(left_pose))

[[ 0.87456155  0.33534923 -0.35040918 -0.5263514 ]
 [-0.33194345  0.9406187   0.0717207   0.02102616]
 [ 0.35363948  0.05358973  0.9339139   0.37803566]
 [ 0.          0.          0.          1.        ]]


- The dot product tells you what amount of one vector goes in the direction of another. 

In [7]:
np.dot(right_pose, inv(left_pose))

array([[ 0.9997334 ,  0.02308935, -0.00166941, -0.07349059],
       [-0.02311522,  0.99957764, -0.01766039, -0.02184239],
       [ 0.00126078,  0.01769348,  0.9998439 ,  0.06695917],
       [ 0.        ,  0.        ,  0.        ,  1.        ]],
      dtype=float32)

In [8]:
x = np.zeros((3, 4, 5))
print(x)
print(np.moveaxis(x, 0, -1).shape)
print(np.moveaxis(x, -1, 0))
# print(np.moveaxis(x, 0, -1))

[[[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]]
(4, 5, 3)
[[[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]]


In [9]:
print(n_img.shape)
print(np.moveaxis(n_img, -1, 0).shape)

(480, 640, 3)
(3, 480, 640)


In [10]:
n_img

array([[[ 95,  92,  87],
        [ 95,  94,  86],
        [ 94,  91,  85],
        ...,
        [179, 170, 171],
        [178, 167, 171],
        [178, 167, 171]],

       [[ 95,  92,  87],
        [ 95,  91,  85],
        [ 94,  88,  84],
        ...,
        [179, 167, 172],
        [178, 166, 171],
        [178, 169, 171]],

       [[ 97,  90,  87],
        [ 97,  90,  85],
        [ 97,  90,  83],
        ...,
        [177, 165, 174],
        [177, 168, 171],
        [177, 174, 171]],

       ...,

       [[128, 116, 116],
        [128, 109, 109],
        [121, 115, 101],
        ...,
        [ 48,  44,  59],
        [ 49,  44,  59],
        [ 51,  45,  59]],

       [[120, 101, 124],
        [120,  94, 112],
        [119,  96, 100],
        ...,
        [ 47,  44,  57],
        [ 48,  44,  57],
        [ 50,  45,  57]],

       [[112,  94, 124],
        [112,  92, 112],
        [117,  89, 100],
        ...,
        [ 47,  44,  57],
        [ 48,  46,  57],
        [ 49,  45,  57]]

In [11]:
x = np.array([1, 2])
y = np.expand_dims(x, axis=0)
print(x)
print(y)

[1 2]
[[1 2]]


In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable


def down_conv_layer(input_channels, output_channels, kernel_size):
    return nn.Sequential(
        nn.Conv2d(
            input_channels,
            output_channels,
            kernel_size,
            padding=(kernel_size - 1) // 2,
            stride=1,
            bias=False),
   nn.BatchNorm2d(output_channels),
   nn.ReLU(),
        nn.Conv2d(
            output_channels,
            output_channels,
            kernel_size,
            padding=(kernel_size - 1) // 2,
            stride=2,
            bias=False),
   nn.BatchNorm2d(output_channels),
   nn.ReLU())

def conv_layer(input_channels, output_channels, kernel_size):
    return nn.Sequential(
        nn.Conv2d(
            input_channels,
            output_channels,
            kernel_size,
            padding=(kernel_size - 1) // 2,
            bias=False),
  nn.BatchNorm2d(output_channels),
        nn.ReLU())

def depth_layer(input_channels):
    return nn.Sequential(
        nn.Conv2d(input_channels, 1, 3, padding=1), nn.Sigmoid())

def refine_layer(input_channels):
    return nn.Conv2d(input_channels, 1, 3, padding=1)

def up_conv_layer(input_channels, output_channels, kernel_size):
    return nn.Sequential(
        nn.Upsample(scale_factor=2, mode='bilinear'),
        nn.Conv2d(
            input_channels,
            output_channels,
            kernel_size,
            padding=(kernel_size - 1) // 2,
            bias=False),
  nn.BatchNorm2d(output_channels),
        nn.ReLU())

def get_trainable_number(variable):
    num = 1
    shape = list(variable.shape)
    for i in shape:
        num *= i
    return num

class enCoder(nn.Module):

    def __init__(self):
        super(enCoder, self).__init__()

        self.conv1 = down_conv_layer(67, 128, 7)
        self.conv2 = down_conv_layer(128, 256, 5)
        self.conv3 = down_conv_layer(256, 512, 3)
        self.conv4 = down_conv_layer(512, 512, 3)
        self.conv5 = down_conv_layer(512, 512, 3)


    def getVolume(self, left_image, right_image, KRKiUV_T, KT_T):

        idepth_base = 1.0 / 50.0
        idepth_step = (1.0 / 0.5 - 1.0 / 50.0) / 63.0

        costvolume = Variable(
            torch.FloatTensor(left_image.shape[0], 64,
                                   left_image.shape[2], left_image.shape[3]))

        image_height = 256
        image_width = 320
        batch_number = left_image.shape[0]

        normalize_base = torch.FloatTensor(
            [image_width / 2.0, image_height / 2.0])

        normalize_base = normalize_base.unsqueeze(0).unsqueeze(-1)

        for depth_i in range(64):
            this_depth = 1.0 / (idepth_base + depth_i * idepth_step)
            transformed = KRKiUV_T * this_depth + KT_T
            demon = transformed[:, 2, :].unsqueeze(1)  
            warp_uv = transformed[:, 0: 2, :] / (demon + 1e-6)
            warp_uv = (warp_uv - normalize_base) / normalize_base
            warp_uv = warp_uv.view(
                batch_number, 2, image_width,
                image_height) 

            warp_uv = Variable(warp_uv.permute(
                0, 3, 2, 1))  
            warped = F.grid_sample(right_image, warp_uv)

            costvolume[:, depth_i, :, :] = torch.sum(
                torch.abs(warped - left_image), dim=1)
        return costvolume

    def forward(self, left_image, right_image, KRKiUV_T, KT_T):
        plane_sweep_volume = self.getVolume(left_image, right_image, KRKiUV_T, KT_T)

        x = torch.cat((left_image, plane_sweep_volume), 1)

        conv1 = self.conv1(x)
        conv2 = self.conv2(conv1)
        conv3 = self.conv3(conv2)
        conv4 = self.conv4(conv3)

        conv5 = self.conv5(conv4)


        return [conv5, conv4, conv3, conv2, conv1]


In [13]:

def encoder_forward(r_img,n_img, r_pose,n_pose, K):
    
    # Left and right image
    left_image = r_img
    right_image = n_img

    # Left and right poses
    left_pose = r_pose
    right_pose = n_pose
    # Camera intrinsic matrix
    camera_k = K

    # Finds how one vector component on the other. Mostly for similarity
    left2right = np.dot(right_pose, inv(left_pose))

    # scale to 320x256
    # Find original width and height
    original_width = left_image.shape[1]
    original_height = left_image.shape[0]
    # Find the scaling factor to the standardized image size
    factor_x = 320.0 / original_width
    factor_y = 256.0 / original_height
    
    # Resize the image to (320, 256)
    left_image = cv2.resize(left_image, (320, 256))
    right_image = cv2.resize(right_image, (320, 256))
    
    # left and right image with dimension (320, 256)
    
    # Take the first row and multiply the first row with factor_x, scaling value
    camera_k[0, :] *= factor_x
    # Take the second row and multiply the second row with factor_y, scaling value
    camera_k[1, :] *= factor_y

    # convert to torch
    # Change the axis
    torch_left_image = np.moveaxis(left_image, -1, 0)
    # torch_left_image dimension (3, 256, 320)
    
    #Expand the dimension
    torch_left_image = np.expand_dims(torch_left_image, 0)
    # torch_left_image (1, 3, 256, 320)
    
    torch_left_image = (torch_left_image - 81.0)/ 35.0
    
    torch_right_image = np.moveaxis(right_image, -1, 0)
    # torch_right_image dimension (3, 256, 320)
    torch_right_image = np.expand_dims(torch_right_image, 0)
    # torch_right_image dimension (1, 3, 256, 320)
    torch_right_image = (torch_right_image - 81.0)/ 35.0

    # Transform the image to tensor 
    left_image_cuda = Tensor(torch_left_image)
    # set the input to a network to volatile if you are doing inference only 
    # and won't be running backpropagation in order to conserve memory.
    left_image_cuda = Variable(left_image_cuda, volatile=True)

    right_image_cuda = Tensor(torch_right_image)
    right_image_cuda = Variable(right_image_cuda, volatile=True)
    
    # take 0 to 3rd row with only the 4th column
    # Translation matrix
    left_in_right_T = left2right[0:3, 3]
    # take 3x3 matrix
    # Rotation matrix
    left_in_right_R = left2right[0:3, 0:3]
    # Take camera matrix
    K = camera_k
    # Take the inverse of camera matrix
    K_inverse = inv(K)
    # Dot product between 3x3 matrix and inversed camera matrix
    KRK_i = K.dot(left_in_right_R.dot(K_inverse))
    # Take the dot product between the matrix and pixel coordinate
    KRKiUV = KRK_i.dot(pixel_coordinate)
    # Dot product between the camera matrix and left in right matrix transalation vector
    KT = K.dot(left_in_right_T)
    # Expand the dimension
    KT = np.expand_dims(KT, -1)
    KT = np.expand_dims(KT, 0)
    # Transform it to floating value
    KT = KT.astype(np.float32)
    KRKiUV = KRKiUV.astype(np.float32)
    KRKiUV = np.expand_dims(KRKiUV, 0)
    # Transform the values to cuda tensor type
    KRKiUV_cuda_T = Tensor(KRKiUV)
    KT_cuda_T = Tensor(KT)
    
    pretrained_encoder = "/home/latai/Documents/Master_thesis/Tutorials/encoder_model_best.pth.tar"
    encoder = enCoder()
    encoder = torch.nn.DataParallel(encoder)
    weights = torch.load(pretrained_encoder, map_location=torch.device('cpu'))
    encoder.load_state_dict(weights['state_dict'])
    encoder.eval()

    conv5, conv4, conv3, conv2, conv1= encoder(left_image_cuda, right_image_cuda, KRKiUV_cuda_T,KT_cuda_T)

#     return conv5, conv4, conv3, conv2, conv1
    return conv5, conv4, conv3, conv2, conv1, left_image_cuda, right_image_cuda, KRKiUV_cuda_T,KT_cuda_T
#     return None

In [14]:
conv5, conv4, conv3, conv2, conv1, left_image_cuda, right_image_cuda, KRKiUV_cuda_T,KT_cuda_T = encoder_forward(r_img,n_img, r_pose,n_pose, K)

/home/latai/anaconda3/envs/latai/lib/python3.7/site-packages/ipykernel_launcher.py:56: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/home/latai/anaconda3/envs/latai/lib/python3.7/site-packages/ipykernel_launcher.py:59: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/home/latai/anaconda3/envs/latai/lib/python3.7/site-packages/torch/nn/functional.py:4194: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  "Default grid_sample and affine_grid behavior has changed "


## Cost volume

In [15]:
left_image = r_img
print(left_image.shape[0])
print(left_image.shape[1])
print(left_image.shape[2])
print(left_image.shape[3])


480
640
3


IndexError: tuple index out of range

In [16]:
def getVolume(left_image, right_image, KRKiUV_T, KT_T):
    
    # ideal base depth
    idepth_base = 1.0 / 50.0
    # Move from 0.5m to maximum of 50m hypothesized depth such that the hypothesized planes are 64
    idepth_step = (1.0 / 0.5 - 1.0 / 50.0) / 63.0
    
    # Didn't understand this part
    costvolume = Variable(
        torch.FloatTensor(left_image.shape[0], 64,
                               left_image.shape[2], left_image.shape[3]))

    image_height = 256
    image_width = 320
    batch_number = left_image.shape[0]
    
    # Normalize the image with certain width and height
    normalize_base = torch.FloatTensor(
        [image_width / 2.0, image_height / 2.0])
    
    # Normalize the image
    normalize_base = normalize_base.unsqueeze(0).unsqueeze(-1)

    for depth_i in range(64):
        this_depth = 1.0 / (idepth_base + depth_i * idepth_step)
        
        print(KRKiUV_T)
        transformed = KRKiUV_T * this_depth + KT_T
        
        demon = transformed[:, 2, :].unsqueeze(1)
        
        warp_uv = transformed[:, 0: 2, :] / (demon + 1e-6)
        
        warp_uv = (warp_uv - normalize_base) / normalize_base
        
        warp_uv = warp_uv.view(
            batch_number, 2, image_width,
            image_height) 
        
        warp_uv = Variable(warp_uv.permute(
            0, 3, 2, 1))
        
        warped = F.grid_sample(right_image, warp_uv)
            
        costvolume[:, depth_i, :, :] = torch.sum(
            torch.abs(warped - left_image), dim=1)
    print("left")   
    print("This depth:", this_depth)
    print("KRKiUV_T:", KRKiUV_T.shape)
    print("KT_T:", KT_T.shape)
    print("transformed:",transformed.shape)
    print("demon", demon.shape)
    print("warp_uv:", warp_uv.shape)
    print("warped", warped.shape)
    print("costvolume", costvolume.shape)
    
    return costvolume

In [17]:
getVolume(left_image_cuda, right_image_cuda, KRKiUV_cuda_T,KT_cuda_T)

tensor([[[ -4.5131,  -4.4824,  -4.4517,  ..., 322.3940, 322.4247, 322.4554],
         [ -2.5484,  -1.5415,  -0.5347,  ..., 244.4919, 245.4988, 246.5056],
         [  0.9919,   0.9920,   0.9920,  ...,   1.0076,   1.0077,   1.0077]]])
tensor([[[ -4.5131,  -4.4824,  -4.4517,  ..., 322.3940, 322.4247, 322.4554],
         [ -2.5484,  -1.5415,  -0.5347,  ..., 244.4919, 245.4988, 246.5056],
         [  0.9919,   0.9920,   0.9920,  ...,   1.0076,   1.0077,   1.0077]]])
tensor([[[ -4.5131,  -4.4824,  -4.4517,  ..., 322.3940, 322.4247, 322.4554],
         [ -2.5484,  -1.5415,  -0.5347,  ..., 244.4919, 245.4988, 246.5056],
         [  0.9919,   0.9920,   0.9920,  ...,   1.0076,   1.0077,   1.0077]]])
tensor([[[ -4.5131,  -4.4824,  -4.4517,  ..., 322.3940, 322.4247, 322.4554],
         [ -2.5484,  -1.5415,  -0.5347,  ..., 244.4919, 245.4988, 246.5056],
         [  0.9919,   0.9920,   0.9920,  ...,   1.0076,   1.0077,   1.0077]]])
tensor([[[ -4.5131,  -4.4824,  -4.4517,  ..., 322.3940, 322.4247, 32

tensor([[[[1.4286, 1.4571, 1.2286,  ..., 6.4857, 6.5429, 6.3714],
          [1.1429, 1.1714, 1.1143,  ..., 6.6286, 6.7714, 6.7429],
          [1.2286, 1.1429, 1.1143,  ..., 6.8286, 6.8857, 6.9714],
          ...,
          [0.2682, 0.2408, 1.1851,  ..., 0.4909, 0.6922, 1.1583],
          [0.5825, 1.1345, 1.8884,  ..., 2.1074, 1.1669, 0.5891],
          [1.6801, 1.9008, 1.3072,  ..., 7.3109, 5.2035, 3.1958]],

         [[1.4286, 1.4571, 1.2286,  ..., 6.4857, 6.5429, 6.3714],
          [1.1429, 1.1714, 1.1143,  ..., 6.6286, 6.7714, 6.7429],
          [1.2286, 1.1429, 1.1143,  ..., 6.8286, 6.8857, 6.9714],
          ...,
          [0.5596, 0.1792, 1.4232,  ..., 0.6074, 0.6525, 1.3457],
          [0.6276, 1.3507, 2.3749,  ..., 2.0556, 1.0582, 0.3920],
          [1.7353, 2.4194, 1.3259,  ..., 7.1970, 5.1202, 3.2227]],

         [[1.4286, 1.4571, 1.2286,  ..., 6.4857, 6.5429, 6.3714],
          [1.1429, 1.1714, 1.1143,  ..., 6.6286, 6.7714, 6.7429],
          [1.2286, 1.1429, 1.1143,  ..., 6

In [18]:
# ideal base depth
idepth_base = 1.0 / 50.0
# Move from 0.5m to maximum of 50m hypothesized depth such that the hypothesized planes are 64
idepth_step = (1.0 / 0.5 - 1.0 / 50.0) / 63.0

In [19]:
print(idepth_base)
print(idepth_step)

print(idepth_step*64)

0.02
0.03142857142857143
2.0114285714285716


In [20]:
def genDistM(poses):
    n = len(poses)
    D = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            D[i, j] = pose_distance(poses[i], poses[j])
    return D

def pose_distance(p1, p2):
    rel_pose = np.dot(p1, inv(p2))
    R = rel_pose[:3, :3]
    t = rel_pose[:3, 3]

    return round(np.sqrt(np.linalg.norm(t) ** 2 + 2 * (1 - min(3.0, np.matrix.trace(R)) / 3)), 4)

In [21]:
latents = conv5

In [22]:
D = torch.from_numpy(np.expand_dims(genDistM(gt_poses), 0))
# Output of the encoder at conv5 layer
Y = torch.stack(latents, dim=1).cpu()
print(D)

TypeError: stack(): argument 'tensors' (position 1) must be tuple of Tensors, not Tensor

In [25]:
self.root = Path(root)
scene_list_path = self.root / 'train_wo_gtav.txt' if train else self.root / 'val.txt'
self.scenes = [self.root / folder[:-1] for folder in open(scene_list_path)]
self.scenes

for folder in folders_list:
    imgs = sorted(folder.files('*.png'))
 
n = len(imgs)

with torch.no_grad():
    poses = []
    idepths = []
    idepths_after = []
    latents = []
    conv1s = []
    conv2s = []
    conv3s = []
    conv4s = []

    preds = []

    depth_gts = []
    for i in tqdm(range(1, n)):  # start with the 2nd frame

        r_pose = gt_poses[i]
        n_pose = gt_poses[i - 1]

        r_img = cv2.imread(imgs[i])
        n_img = cv2.imread(imgs[i - 1])

        camera_k = np.loadtxt(scene / 'K.txt').astype(np.float32).reshape((3, 3))

        conv5, conv4, conv3, conv2, conv1 = encoder_forward(r_img, n_img, r_pose, n_pose, camera_k)

        poses.append(r_pose)

        latents.append(conv5)
        conv4s.append(conv4)
        conv3s.append(conv3)
        conv2s.append(conv2)
        conv1s.append(conv1)

        gt_depth = np.load(gts[i])
        depth_gts.append(gt_depth)

    D = torch.from_numpy(np.expand_dims(genDistM(poses), 0))
    Y = torch.stack(latents, dim=1).cpu()

#     Z = gplayer(D, Y)

NameError: name 'Path' is not defined

In [28]:
pretrained_encoder = "/home/latai/Documents/Master_thesis/Tutorials/encoder_model_best.pth.tar"

In [29]:
from enCoder import enCoder
pretrained_encoder = "/home/latai/Documents/Master_thesis/Tutorials/encoder_model_best.pth.tar"
encoder = enCoder()
encoder = torch.nn.DataParallel(encoder)
weights = torch.load(pretrained_encoder, map_location=torch.device('cpu'))
encoder.load_state_dict(weights['state_dict'])
encoder.eval()

DataParallel(
  (module): enCoder(
    (conv1): Sequential(
      (0): Conv2d(67, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv2d(128, 128, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
    )
    (conv2): Sequential(
      (0): Conv2d(128, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv2d(256, 256, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), bias=False)
      (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
    )
    (conv3): Sequential(
      (0): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 